In [11]:
import pandas as pd
import chardet
import os
os.add_dll_directory(r'C:\Users\ZZ01KD865\Documents\projects\csv_checker\env\Lib\site-packages\clidriver\bin') 
import ibm_db
import json
from ftplib import FTP
import pandas as pd
import io
import sys

In [14]:
import zipfile
import os
import pandas as pd
from io import BytesIO

# Specify the directory containing the zip files
directory = './compress/'

# List to store DataFrames from CSV files
dataframes = []

# Iterate over each file in the directory
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    # Check if the file is a zip file
    if filename.endswith('.zip'):
        # Open the zip file
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            # Iterate over each file in the zip
            for file_info in zip_ref.infolist():
                # Check if the file is a CSV
                if file_info.filename.endswith('.csv'):
                    # Extract the file contents into memory
                    with zip_ref.open(file_info) as csv_file:
                        # Read the CSV data into a DataFrame
                        df = pd.read_csv(BytesIO(csv_file.read()))
                        # Append the DataFrame to the list
                        dataframes.append(df)
                        print(f"{file_info.filename} read successfully.")

# Now you have a list of DataFrames from all the CSV files
# You can concatenate them if needed
result_df = pd.concat(dataframes, ignore_index=True)

data.csv read successfully.


In [15]:
def get_csv_file_size(file_path):
    try:
        # Get the size of the file in bytes
        file_size_bytes = os.path.getsize(file_path)

        # Convert bytes to kilobytes or megabytes if needed
        file_size_kb = file_size_bytes / 1024  # Convert bytes to kilobytes
        file_size_mb = file_size_kb / 1024     # Convert kilobytes to megabytes

        return file_size_bytes, file_size_kb, file_size_mb

    except Exception as e:
        print(f"Error getting file size: {e}")
        return None
    
# def get_file_size_ftp(ftp, file_path):
#     # Get the size of the file
#     file_size = ftp.size(file_path)
#     # Convert the file size to MB
#     file_size_mb = file_size / (1024 * 1024)
#     return "{:.2f}".format(file_size_mb)


# def get_file_encoding(file_path):
#     try:
#         with open(file_path, 'rb') as f:
#             # Read a chunk of the file to detect the encoding
#             rawdata = f.read()
#             result = chardet.detect(rawdata)
#             return result['encoding']
#     except Exception as e:
#         print(f"Error detecting file encoding: {e}")
#         return None

def get_file_size(df):
    # Get memory usage of each column in bytes
    memory_usage_bytes = df.memory_usage(deep=True).sum()
    # Convert bytes to megabytes (MB)
    memory_usage_mb = memory_usage_bytes / (1024 * 1024)
    return memory_usage_mb


def get_file_encoding(bytes_io):
    try:
        bytes_io.seek(0)
        rawdata = bytes_io.read()
        result = chardet.detect(rawdata)
        return result['encoding']
    except Exception as e:
        print(f"Error detecting file encoding: {e}")
        return None

def extract_filename(file_path):
    return os.path.basename(file_path)


def get_csv_metadata(file_path, df):
    try:
        df = pd.read_csv(file_path)
        column_names = df.columns.tolist()
    
        data_types = df.dtypes.tolist()
        
        # summary_statistics = df.describe()
        
        num_rows, num_columns = df.shape
        metadata = {
            # 'column_names': column_names,
            # 'data_types': data_types,
            # 'summary_statistics': summary_statistics,
            'filename': [file_path,],
            # 'encoding':[get_file_encoding(bytes_io),],
            'file_size':[get_file_size(df),],
            'num_rows': [num_rows,],
            'num_columns': [num_columns,]
            
        }
        return pd.DataFrame(metadata)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None



In [16]:
def get_config_object(file_name='config.json'): 
	with open('config.json', 'r') as file: 
		config = json.load(file)
	return config

In [17]:
def store_df_to_db2(df, table_name):
    config = get_config_object()
    # Connection parameters
    database_name = config['database']['database_name']
    hostname = config['database']['host']
    port = config['database']['port']
    protocol = "TCPIP"
    user = config['database']['username']
    password = config['database']['password']

    # Connection string
    dsn = (
        "DRIVER={{IBM DB2 ODBC DRIVER}};"
        "DATABASE={0};"
        "HOSTNAME={1};"
        "PORT={2};"
        "PROTOCOL={3};"
        "UID={4};"
        "PWD={5};"
    ).format(database_name, hostname, port, protocol, user, password)

    # Establish the database connection
    conn = ibm_db.connect(dsn, "", "")

    # Check if the connection is successful
    if conn:
        print("Connected to the database")

        # Insert DataFrame into DB2 table
        df.to_sql(table_name, conn, if_exists='append', index=False)

        # Close the connection
        ibm_db.close(conn)
        print("Connection closed")
    else:
        print("Failed to connect to the database")

In [20]:
def get_df_from_zip(file_bytes):
    dataframes = []
    file_bytes.seek(0)
    with zipfile.ZipFile(file_bytes,'r') as zip_ref:
        # Iterate over each file in the zip
        for file_info in zip_ref.infolist():
                # Check if the file is a CSV
                if file_info.filename.endswith('.csv'):
                    # Extract the file contents into memory
                    with zip_ref.open(file_info) as csv_file:
                        # Read the CSV data into a DataFrame
                        df = pd.read_csv(BytesIO(csv_file.read()))
                        # Append the DataFrame to the list
                        dataframes.append(df)
        return pd.concat(dataframes, ignore_index=True)

def read_csv_from_ftp_and_delete(config_obj):
    try:
        host = config_obj['ftp']['host']
        username = config_obj['ftp']['username']
        password = config_obj['ftp']['password']
        directory = config_obj['ftp']['directory']
        df_data_list = []
        df_meta_list = []
        # Connect to the FTP server
        ftp = FTP(host)
        ftp.login(username, password)
        # Change to the specified directory

        ftp.cwd(directory)

        # List all files in the directory
        file_list = ftp.nlst()

        for filename in file_list:
            # Read the CSV file into a BytesIO object
            zip_data = io.BytesIO()
            ftp.retrbinary(f'RETR {filename}', zip_data.write)
            zip_data.seek(0)  # Reset the file pointer to the beginning

            #convert zip file to DataFrame
            # Read the CSV data from BytesIO into a pandas DataFrame
            df_data = get_df_from_zip(zip_data)

            # df_data = pd.read_csv(csv_data)
            # df_meta = get_csv_metadata(ftp,filename,csv_data)
            df_meta = get_csv_metadata(ftp,filename,df_data)
            df_data_list.append(df_data)
            df_meta_list.append(df_meta)

            # Delete the file on the remote machine
            ftp.delete(filename)
            print(f"File '{filename}' deleted successfully.")

        # Close the FTP connection
        ftp.quit()
        return df_data_list, df_meta_list

    except Exception as e:
        print(f"Error: {e}")
        return None, None

Error: [Errno 11001] getaddrinfo failed


In [ ]:
def main():
    args = sys.argv
    try: 
        config_path = args[1]
        config_obj = get_config_object(config_path)
        # get meta df and data df
        df_data, df_meta = read_csv_from_ftp_and_delete(config_obj)
        # store df_data
        store_df_to_db2(df_data,config_obj['database']['meta_table'])
        # store df_meta
        store_df_to_db2(df_meta, config_obj['database']['data_table'])
    except:
        print("config file doesn't exsit")
